In [1]:
%run _startup.ipynb

In [2]:
import requests
from lxml import etree
from lxml.html import fromstring, tostring
from bs4 import BeautifulSoup

### 1. Parse all FOMC meetings

In [3]:
# Parse all meetings for the last 5 years
url = f'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
resp = requests.get(url)

soup = BeautifulSoup(resp.text, "html.parser")
cards = soup.findAll('div', class_='col-xs-12 col-md-4 col-lg-2')

list0 = list()
list1 = list()
for card in cards:
    a = ''.join([str(a1) for a1 in card.contents])
    b = re.findall('href="/newsevents.+a.htm', a)
    if len(b) > 1:
        print(b)
    if len(b) == 0:
        continue
    list0.append(a)
    list1.append(b[0][-13:-5])

df1 = pd.DataFrame({'dt1': list1})

In [4]:
# Parse historical meetings
list_resp = list()
for year in tqdm(range(2018,1980,-1)):
    url = f'https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm'
    resp = requests.get(url)
    list_resp.append(resp)
    time.sleep(3)
# list_resp

list2 = list()
for resp in list_resp:
    soup = BeautifulSoup(resp.text, "html.parser")
    cards = soup.findAll('div', class_='col-xs-12 col-md-6')
    
    list0 = list()
    list1 = list()
    for card in cards:
        a = ''.join([str(a1) for a1 in card.contents])
        b = a.split('.pdf')[0].split('/')[-1]
        c = re.findall('[0-9]+',b)
        if len(c) == 0:
            print(b)
            continue
        list1.append(c[0])
    list2+=list1

df2 = pd.DataFrame({'dt1': list2})

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [03:04<00:00,  4.86s/it]


p>



In [8]:
# merge and aggregate
df = pd.concat([df1, df2])
df.drop_duplicates(inplace = True)

df['dt_month'] = pd.to_datetime(df['dt1'].str[:6]+'01')
df = df.groupby('dt_month').dt1.agg([len,lambda x: ','.join(x)]).reset_index()
df.columns = ['dt_month','cnt_meetings','meetings']

df.to_pickle('data/df_meetings.pkl')

print(df.shape)
df[:3]

(370, 3)


,dt_month,cnt_meetings,meetings
0,1981-02-01,2,"19810202,19810203"
1,1981-03-01,1,19810331
2,1981-05-01,1,19810518


### 2. Parse statements from the last FOMC meetings

In [11]:
# Parse all meetings for the last 5 years
url = f'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
resp = requests.get(url)

soup = BeautifulSoup(resp.text, "html.parser")
cards = soup.findAll('div', class_='col-xs-12 col-md-4 col-lg-2')

list0 = list()
list1 = list()

for card in cards:
    a = ''.join([str(a1) for a1 in card.contents])
    b = re.findall('href="/newsevents.+a.htm', a)
    if len(b) > 1:
        print(b)
    if len(b) == 0:
        continue
    list0.append(a)
    list1.append(b[0][6:])

df = pd.DataFrame({
    'card': list0, 
    'url': list1
})
df['dt1'] = pd.to_datetime(df['url'].str[34:42])

In [12]:
# Number of statements per year:
df.groupby('dt1').size().resample('Y').sum().sort_index()

dt1
2019-12-31     9
2020-12-31    12
2021-12-31     8
2022-12-31     8
2023-12-31     8
2024-12-31     7
Freq: A-DEC, dtype: int64

In [14]:
# Parse statements
url1 = 'https://www.federalreserve.gov'
list_resp = list()
for url2 in tqdm(df['url']):
    url = url1 + url2
    resp = requests.get(url)
    list_resp.append(resp)
    time.sleep(5)
df['resp'] = list_resp

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [05:56<00:00,  6.85s/it]


In [15]:
# Get text from statements
list_txt = list()
for resp in df['resp']:
    soup = BeautifulSoup(resp.text, "html.parser")
    cards = soup.findAll('div', class_='col-xs-12 col-sm-8 col-md-8')
    if len(cards) != 1:
        print(cards)   
    a = ''.join([str(a1) for a1 in cards[0].get_text()])
    list_txt.append(a)
df['txt'] = list_txt
df = df.sort_values('dt1').reset_index(drop = True)

df.to_pickle('data/df_statements.pkl')

print(df.shape)
df[:3]

(52, 5)


,card,url,dt1,resp,txt
0,"\n<strong>Statement:</strong><br/>\n<a href=""/...",/newsevents/pressreleases/monetary20190130a.htm,2019-01-30,<Response [200]>,\nInformation received since the Federal Open ...
1,"\n<strong>Statement:</strong><br/>\n<a href=""/...",/newsevents/pressreleases/monetary20190320a.htm,2019-03-20,<Response [200]>,\nInformation received since the Federal Open ...
2,"\n<strong>Statement:</strong><br/>\n<a href=""/...",/newsevents/pressreleases/monetary20190501a.htm,2019-05-01,<Response [200]>,\nInformation received since the Federal Open ...
